In [3]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [23]:
# ==============================================================================
# PROGRAM INFERENSI FINAL (LOGIKA CELL 10 + STRICT + NASYA FIX)
#
# KEGUNAAN: Membaca Folder Test -> MTCNN (Raw) -> Rumus Cell 10 -> CSV
# SPESIAL :  "Nasya Aulia Efendi" menjadi Index 0.
# ==============================================================================

import torch
import torch.nn as nn
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image, ImageOps
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from google.colab import drive

# 1. SETUP DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Menggunakan perangkat: {device}')

# ==============================================================================
# KONFIGURASI
# ==============================================================================
EXTERNAL_TEST_DIR   = '/Test'
MODEL_FILENAME      = 'Model FaceNet.pth'
CSV_OUTPUT_FILENAME = 'predict.csv'
TARGET_CLASS_ZERO   = 'Nasya Aulia Efendi' # <--- Nama Folder Nasya (Sesuaikan Huruf Besar/Kecil)
# ==============================================================================

# Cek Path
if not os.path.exists(EXTERNAL_TEST_DIR):
    raise FileNotFoundError(f" Folder Test tidak ditemukan di: {EXTERNAL_TEST_DIR}")

# Cari model otomatis
if not os.path.exists(MODEL_FILENAME):
    alt_path = f'/content/drive/MyDrive/Dataset_Wajah/{MODEL_FILENAME}'
    if os.path.exists(alt_path):
        MODEL_FILENAME = alt_path
        print(f" Model ditemukan di: {MODEL_FILENAME}")
    else:
        raise FileNotFoundError(f" File Model '{MODEL_FILENAME}' tidak ditemukan!")

# 3. SETUP MTCNN (Raw Mode - Output 0-255)
print("Menyiapkan MTCNN...")
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709,
    post_process=False,
    device=device
)

# ------------------------------------------------------------------------------
# 4. DAPATKAN NAMA KELAS & MANIPULASI URUTAN (NASYA = 0)
# ------------------------------------------------------------------------------
# A. Ambil semua folder & Urutkan Abjad dulu
raw_folders = os.listdir(EXTERNAL_TEST_DIR)
folder_names = sorted([d for d in raw_folders if os.path.isdir(os.path.join(EXTERNAL_TEST_DIR, d)) and not d.startswith('.')])

# B. PINDAHKAN NASYA KE DEPAN
if TARGET_CLASS_ZERO in folder_names:
    print(f" Ditemukan '{TARGET_CLASS_ZERO}'. Memindahkan ke Index 0...")
    folder_names.remove(TARGET_CLASS_ZERO) # Hapus dari posisi lama
    folder_names.insert(0, TARGET_CLASS_ZERO) # Masukkan ke posisi pertama
else:
    print(f" PERINGATAN: Folder '{TARGET_CLASS_ZERO}' tidak ditemukan! Menggunakan urutan abjad biasa.")

# C. Buat Mapping Baru
class_to_idx = {cls_name: i for i, cls_name in enumerate(folder_names)}
idx_to_class = {i: cls_name for i, cls_name in enumerate(folder_names)}
N_CLASSES = len(folder_names)

print(f"Total Kelas: {N_CLASSES}")
print(f"Index 0 Sekarang: {folder_names[0]}")     # Harus Nasya
print(f"Index 1 Sekarang: {folder_names[1]}")     # Harus Abraham (atau urutan abjad selanjutnya)

# ------------------------------------------------------------------------------

# 5. LOAD MODEL (PERSIS CELL 10 + STRICT TRUE)
print("Memuat Model...")

# Inisialisasi persis seperti Cell 10 Anda:
model = InceptionResnetV1(pretrained=None, classify=True, num_classes=N_CLASSES).to(device)

try:
    model.load_state_dict(torch.load(MODEL_FILENAME, map_location=device), strict=True)
    print(" Bobot Model Berhasil Dimuat Sempurna (Strict Mode).")
except RuntimeError as e:
    print("\n KRITIS: JUMLAH KELAS TIDAK COCOK!")
    print("Error Detail:", e)
    raise e

model.eval()

# 6. LOOPING PROSES (LOGIKA CELL 10)
y_true = []
y_pred = []
csv_filenames = []
csv_labels = []
csv_probs = []

print("\nMemulai Prediksi...")

# Kita loop berdasarkan folder_names yang SUDAH DIMODIFIKASI urutannya
for class_name in tqdm(folder_names, desc="Kelas"):
    class_dir = os.path.join(EXTERNAL_TEST_DIR, class_name)
    class_idx = class_to_idx[class_name] # Nasya akan dapat ID 0 disini

    for img_file in os.listdir(class_dir):
        if not img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp', '.heic')):
            continue

        img_path = os.path.join(class_dir, img_file)

        try:
            # A. Buka Gambar
            img = Image.open(img_path).convert('RGB')
            img = ImageOps.exif_transpose(img)

            # B. Deteksi Wajah (Output 0-255)
            face_tensor = mtcnn(img)

            if face_tensor is not None:
                # =======================================================
                # C. LOGIKA CELL 10 (MATEMATIKA)
                # =======================================================

                # 1. Normalisasi Manual
                face_tensor = face_tensor / 255.0

                # 2. Standardisasi
                face_input = fixed_image_standardization(face_tensor)

                # 3. Batch Dimension
                face_input = face_input.unsqueeze(0).to(device)

                # D. Prediksi
                with torch.no_grad():
                    logits = model(face_input)
                    probabilities = torch.nn.functional.softmax(logits, dim=1)
                    conf, preds = torch.max(probabilities, 1)

                pred_idx = preds.item()
                conf_score = conf.item() * 100

                # Simpan
                y_true.append(class_idx)
                y_pred.append(pred_idx)

                csv_filenames.append(img_file)
                csv_labels.append(idx_to_class[pred_idx])
                csv_probs.append(f"{conf_score:.2f}%")

        except Exception as e:
            pass

# 7. EVALUASI
print("\n" + "="*70)
if len(y_true) > 0:
    acc = accuracy_score(y_true, y_pred)
    print(f" HASIL AKURASI FINAL: {acc*100:.2f}%")
    print("="*70)
    print(classification_report(y_true, y_pred, target_names=folder_names, zero_division=0))
else:
    print("Tidak ada data yang berhasil diolah.")

# 8. SIMPAN CSV
df = pd.DataFrame({
    'filename': csv_filenames,
    'prediction': csv_labels,
    'confidence': csv_probs
})
df.to_csv(CSV_OUTPUT_FILENAME, index=False)
print(f"\n File CSV tersimpan: {CSV_OUTPUT_FILENAME}")

Menggunakan perangkat: cpu
Menyiapkan MTCNN...
⚠️ Ditemukan 'Nasya Aulia Efendi'. Memindahkan ke Index 0...
Total Kelas: 70
Index 0 Sekarang: Nasya Aulia Efendi
Index 1 Sekarang: Abraham Ganda Napitu
Memuat Model...
✅ Bobot Model Berhasil Dimuat Sempurna (Strict Mode).

Memulai Prediksi...


Kelas:   0%|          | 0/70 [00:00<?, ?it/s]


 HASIL AKURASI FINAL: 97.14%
                                 precision    recall  f1-score   support

             Nasya Aulia Efendi       1.00      1.00      1.00         1
           Abraham Ganda Napitu       1.00      1.00      1.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       1.00      1.00      1.00         1
                   Aldi Sanjaya       1.00      1.00      1.00         1
                        Alfajar       1.00      1.00      1.00         1
            Alief Fathur Rahman       1.00      1.00      1.00         1
 Arkan Hariz Chandrawinata Liem       1.00      1.00      1.00         1
               Bayu Ega Ferdana       1.00      1.00      1.00         1
          Bayu Prameswara Haris       1.00      1.00      1.00         1
           Bezalel Samuel Manik       1.00      1.00      1.00         1
           Bintang Fikri Fauzan       1.00      1.00      1.00         1
              Boy Sa